In [2]:
#!pip install sklearn --upgrade
#!pip install imblearn --upgrade

In [3]:
import pandas               as pd
#import matplotlib.pyplot    as plt

from sklearn.model_selection  import train_test_split, GridSearchCV,\
                                     StratifiedKFold
from sklearn.preprocessing    import OneHotEncoder
from sklearn.metrics          import balanced_accuracy_score,\
                                     confusion_matrix, roc_auc_score
# Import xgboost
import xgboost              as xgb

from imblearn.over_sampling   import SMOTE
from imblearn.under_sampling  import RandomUnderSampler
from imblearn.pipeline        import Pipeline

In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
#client_7463ca614953494181369d0b1ae8c0f2 = ibm_boto3.client(service_name='s3',
#    ibm_api_key_id='hh_MYLG-3Hq-R7hcNvajeJcBLBiWFFT49huB1lJerY5s',
#    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
#    config=Config(signature_version='oauth'),
#    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

#body = client_7463ca614953494181369d0b1ae8c0f2.get_object(Bucket='algar-donotdelete-pr-oazmap8qrzxynm',Key='algar-dataset-treino.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
#if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

#df = pd.read_csv(body)
#df_data_1.head()


ModuleNotFoundError: No module named 'botocore'

In [ ]:
# Columns ith one value
a= [col for col in df.columns if df[col].value_counts().count()<2]
print('Columns with unique value:', a)

In [ ]:
# Subordinado has unique values
print(df['Subordinado'].value_counts().count() == len(df['Subordinado']))

In [ ]:
# Get Features and target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# Eliminate columns with unique feature values
X = X.drop(a + ['Subordinado'], axis=1)

# Categorical Columns
c_columns = X.select_dtypes(include='object')
nc_columns = [col for col in X.columns if col not in c_columns.columns]

ohe = OneHotEncoder(handle_unknown='ignore')

c_columns = pd.DataFrame(ohe.fit_transform(c_columns).toarray())
c_columns.columns = ohe.get_feature_names()

X = pd.concat([X[nc_columns], c_columns], axis=1)

In [ ]:
# Train test split  75% / 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=555)

In [ ]:
# Assemble Model

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', random_state=123)

# tunning parameters
gbm_param_grid = [
    {
     'SMOTE__sampling_strategy': [0.5, 0.4, 0.3],
     'under__sampling_strategy': [0.5, 0.6, 0.7],
     'model__colsample_bytree': [0.055],
     'model__n_estimators': [130, 150, 200],
     'model__max_depth': [1, 2, 3],
     'model__reg_alpha': [0, 0.25, 0.3, 0.35],
     'model__learning_rate': [0.3, 0.35, 0.5, 0.6]
    }
]
# Instantiate the classifier: gbm
gbm = xgb.XGBClassifier()
over = SMOTE(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=555)
kfold = StratifiedKFold(n_splits=5, shuffle=False)

# Assamble Pipeline
steps = [('SMOTE', over),
         ('under', under), 
         ('model', gbm)]

pipeline = Pipeline(steps=steps)

# Perform grid search: grid_mse
grid_acc = GridSearchCV(param_grid=gbm_param_grid, estimator=pipeline,
                        scoring = "balanced_accuracy", cv=kfold, verbose=1,
                        n_jobs=4)

In [ ]:
#Tune Model
# Fit grid_mse to the data
grid_acc.fit(X_train, y_train)

In [5]:
# Print scores
# Print the best parameters and lowest RMSE
print("\nBest parameters found: ", grid_acc.best_params_)
print("\nHighest average balanced accuracy found: %.4f" % grid_acc.best_score_)

print('\nBest Estimator: \n', grid_acc.best_estimator_)

print(' \nBest test score: %f' % (grid_acc.score(X_test, y_test)))

#Print scores
print('\nTrain score: %f' % grid_acc.score(X_train, y_train))
print('Test  score: %f\n' % grid_acc.score(X_test, y_test))

y_pred = grid_acc.predict(X_test)
ba = balanced_accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=['Não', 'Sim'])
ac = cm[0,0]/(cm[0,0]+cm[0,1])
sp = cm[1,1]/(cm[1,1]+cm[1,0])
auc = roc_auc_score(y_test=='Não', y_pred=='Não')

print('\nBalanced Accuracy : %.4f' % ba)
print('AUC               : %.4f' % auc)
print('Accuracy          : %.4f' % ac)
print('Specificity       : %.4f\n' % sp)

NameError: name 'grid_acc' is not defined